# GOLD on MCX

MCX, the Multi-Commodity eXchange of India, has a number of gold futures contracts listed. This is how you explore them on pluto.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
commoditiesFnO <- CommodityFuturesAndOptions()

In [ ]:
#list all gold contracts on the MCX

goldMcx <- commoditiesFnO$McxEod() %>%
    filter(CONTRACT %like% '%gold%' & (OTYPE == 'XX' | OTYPE == 'FUTCOM')) %>%
    group_by(CONTRACT) %>%
    summarize(ST = min(TIME_STAMP, na.rm=T), ET = max(TIME_STAMP, na.rm=T)) %>%
    arrange(ST) %>%
    collect()

goldMcx %>%
    print(n=Inf)

Of these the first two - GOLD and GOLD MINI - are the oldest and represent gold of 1 kg and 100 grams with 995 purity. GOLD MINI has expirations every month whereas GOLD doesn't. GOLDGUINEA and GOLDPETAL were introduced much later in 2008 and 2011, respectively. We will cycle through these four contracts: GOLD, GOLDM, GOLDGUINEA and GOLDPETAL below.

In [ ]:
expirySeries <- 0 # closest expiry
contractNames <- c('GOLD', 'GOLDM', 'GOLDGUINEA', 'GOLDPETAL')

In [ ]:
cName <- 'GOLD'
px <- commoditiesFnO$McxEod() %>%
    filter(CONTRACT == cName & (OTYPE == 'XX' | OTYPE == 'FUTCOM')) %>%
    select(TIME_STAMP, CLOSE, VOLUME) %>%
    collect()

pXts <- xts(px[,-1], px[,1])
names(pXts) <- c('Close', 'Volume')
chartSeries(pXts, theme = 'white', name = cName, log.scale = T)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)